### Create streams and tables

In [38]:
import os
import v3io_frames as v3f
from v3io import dataplane

def handler(context, event):
    # Create streams
    frames_client = v3f.Client("framesd:8081", container="bigdata")
    frames_client.create("stream",
                         table=os.getenv("RAW_VIDEO_STREAM"),
                         shards=100,
                         retention_hours=24)
    context.log_result('raw_video_stream_url', "http://v3io-webapi:8081/%s/%s/"% (os.getenv('IGZ_CONTAINER'),os.getenv('RAW_VIDEO_STREAM')))
     
    frames_client.create("stream",
                         table=os.getenv("TAGGED_VIDEO_STREAM"),
                         shards=100,
                         retention_hours=24)
    context.log_result('tagged_video_stream_url', "http://v3io-webapi:8081/%s/%s/"% (os.getenv('IGZ_CONTAINER'),os.getenv('TAGGED_VIDEO_STREAM')))

    # Create camera list table
    v3io_client = dataplane.Client(max_connections=1)
    v3io_client.create_schema(container=os.getenv("IGZ_CONTAINER"),
                              path=os.getenv("CAMERA_LIST_TBL"),
                              key='cameraID',
                              fields=[
                                  {'name': 'cameraID', 'type': 'string', 'nullable': False},
                                  {'name': 'shard', 'type': 'long', 'nullable': False},
                                  {'name': 'url', 'type': 'string', 'nullable': False},
                                  {'name': 'active', 'type': 'boolean', 'nullable': False},
                                  {'name': 'feed_method', 'type': 'string', 'nullable': False},
                                  {'name': 'users', 'type': 'string', 'nullable': True},
                                  {'name': 'password', 'type': 'string', 'nullable': True},
                              ])
    
    # Add camera to table
    v3io_client.put_item(container=os.getenv("IGZ_CONTAINER"),
                         path=os.path.join(os.getenv("CAMERA_LIST_TBL"), os.getenv("CAMERA_ID")),
                         attributes={'cameraID': os.getenv("CAMERA_ID"),
                                     'shard': int(os.getenv("SHARD_ID")),
                                     'url': os.getenv("CAMERA_URL"),
                                     'feed_method': "push",
                                     'active': True})